# 3. Decision Tree Algorithm
A  decision tree has a tree structure with decision nodes and the leaf nodes. The decision nodes check a condition and then further branches out whereas the leaf nodes are the final outcomes. 

There are both Regression Decision trees and Classification Decision Trees, which are used when the dependent variable is continuous and categorical respectively.<br>
In Regression Tree, the prediction made by the leaf node is the mean of all observations falling in that region , where as it is the mode in case of Classificatiom Trees.

## 1. Regression Decision Tree:

### Step 1 - Create a Gini Index.
Gini Index is the name of cost function(loss function) to evaluate the splits. Splits are nothing but division of the decision node in further sub nodes.

Conditions for Gini Index:
- It works with Categorical Target Variable.
- It performs only Binary splits ( i.e. divide into two sub-nodes ) 

Git score tells you how good the split is by how mixed the classes are in the two groups created by the split. (groups = sub-nodes). If gini score is 0 , that means the split is perfect and the samples in each group belong to same class. The worst case is when both group contains all the possible classes, where gini score is 0.5.

To calculate Gini index:
- Calculate the proportion of classes in each group .
proportion = $ \frac{no. of class-value}{no. of rows} $ <br>
For example;we have two groups - group1 belonging to class 0 and group2 belonging to class 1, both containing 2 rows.<br>
proportion for:
       - group_1_class_0 = 2 / 2 = 1
       - group_1_class_1 = 0 / 2 = 0
       - group_2_class_0 = 0 / 2 = 0
       - group_2_class_1 = 2 / 2 = 1
- Then we calculate gini_index for each child node.
gini_index = 1.0 - sum(proportion * proportion)
- Then we calculate gini_index for each group by the size of the group, relative to all of the samples in the parent. Samples = total no of rows including all groups.
gini_index = (1.0 - sum(proportion * proportion)) * (group_size/total_samples)

Example; gini_index for :
        - Gini(group_1) = (1 - (1*1 + 0*0)) * 2/4 = 0.0
        - Gini(group_2) = (1 - (0*0 + 1*1)) * 2/4 = 0.0
- Then we calculate Gini Score, which is equal to sum of ginit_index of both groups. 
gini_score = Gini(group_1) + Gini(group_2)

For example; In above case
gini_score = 0 , which is perfect split.

In [9]:
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
        return gini

### Step 2 - Create a Split.
This means to find the index of the attribute that needs to be split and the value by which to split samples(rows) on that attribute.
It contains two steps:
- Splitting a Dataset
- Evaluating all splits

### Splitting a Dataset.
It means to split the rows of the dataset given the index of attribute and the split value for that attribute.<br>
It is done by iterating over each row and checking if attribute value is less or more than the split value and assigning it in left or right group respectively.<br>
Left group has rows whose attribute value is less than the split value.<br>
Right group has rows whose attribute value is more than or equal to the split value.

In [10]:
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

### Evaluating a Split.
In a given dataset, we have to check each value on every attribute as a candidate split and then evaluate the cost of each split and get the best possible split.<br>
The best split is then used as a node in the decision tree.

To find best split, what we do is : <br>
Create a dictionary by the key as a split number and value as the list of the index of the chosen attribute, the value of that attribute by which to split and the two groups of data split by the chosen split point.

In [28]:
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index + 1, 'value':b_value, 'groups':b_groups}

In [29]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
split = get_split(dataset)
print(split)

{'index': 1, 'value': 6.642287351, 'groups': ([[2.771244718, 1.784783929, 0], [1.728571309, 1.169761413, 0], [3.678319846, 2.81281357, 0], [3.961043357, 2.61995032, 0], [2.999208922, 2.209014212, 0]], [[7.497545867, 3.162953546, 1], [9.00220326, 3.339047188, 1], [7.444542326, 0.476683375, 1], [10.12493903, 3.234550982, 1], [6.642287351, 3.319983761, 1]])}


### Step 3 - Build a Tree.
To build a tree, we need to add many nodes in a specific manner.<br>
It is divided in 3 parts :
- Terminal Nodes.
- Recursive Splitting.
- Building a Tree.

### Terminal Nodes
Terminal nodes are the leaf nodes. In this part , we decide when to stop growing the tree.<br>
That can be done by using the depth and the number of rows that the node is responsible for in the training dataset. <br>
- Maximum Tree Depth:
This means maximum number of nodes from the root node. Once we get the maximum get , we must stop adding new nodes.
- Minimum Node Records:
This means minimum number of samples that a given node is responsible for. Once at or below this minimum, we must stop splitting and adding new nodes.<br>
One more approach is to decide when the split at node will belong to one group only, which is possible . By this we will be unable to split the node further and it will become a terminal node which will be used for final prediction.

Final prediction will be made by taking the group of rows assigned to that node and selecting the most common class value in the group.

In [30]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

### Recursive Splitting
When we split a node , our dataset gets divided into groups. For further nodes we have to call __get_split__ function again for the new groups.<br>
The new nodes added to a node are called child nodes of that node. a node may contain zero children(if it is a terminal node)b , 1 child( if one side makes a prediction directly ) and 2 children ( left and right child ). 

This part defines a function which controls when to stop calling the __get_split__ function, i.e. when to stop adding nodes.

The argumnents for this function are - node, maximum depth, minimum number of samples in a node and the current depth of a node.

steps to do it:
- First we split our datastet(root node) into left and right group and then delete the groups from that node, so that we can store new groups in the variable.
- Then we check if either left or right group of rows is empty and if so we create a terminal node using what records we do have.
- Then we check if we reached our maximum depth and if so we create terminal node.
- Then we check for left child, if it has rows less than the minimum size , we create it a terminal node. Otherwise we do the same procedures from step 1 and increase depth by 1.
- Then we do the same process on right child.

In [31]:
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

### Building a Tree
To build a tree , first step is to split the training data and then call recursive splitting function.<br>
And then to print the tree, we recursively print out nodes of the decision tree with one line per node.


In [32]:
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root


In [38]:
def print_tree(node, depth=0):
    if isinstance(node, dict):  # isinstance() function returns True if the specified object is of the specified type, otherwise False
    # It will check if the node is dictionary or not.
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']), node['value'])))
        #Depth*' ' is used for right indenting to show each node and it's child nodes. 
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
        # when the node will become terminal node , it will be integer and then else statement will be executed.
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [36]:
tree = build_tree(dataset, 5, 2)
print_tree(tree)

[X1 < 6.642]
 [X1 < 2.771]
  [0]
  [X1 < 2.771]
   [0]
   [0]
 [X1 < 7.498]
  [1]
  [X1 < 7.498]
   [1]
   [1]


Let's explain the following output as a tree. <br>
`[X1 < 6.642]    # root node (A) 
 [X1 < 2.771]    # Left child node of A (P)
  [0]            # Left Terminal child node of P with outcome as 0
  [X1 < 2.771]   # Right child node of P (X)
   [0]           # Left terminal child node of X
   [0]           # Right terminal child node of X
 [X1 < 7.498]    # Right child node of A (Q) 
  [1]            # Left terminal child node of Q
  [X1 < 7.498]   # Right child node of Q (Y)
   [1]           # Left terminal child node of Y
   [1]           # Right terminal child node of Y.
`


### Step 4 - Making a Prediction.


In [39]:
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

In [40]:
stump = {'index': 0, 'right': 1, 'value': 6.642287351, 'left': 0}
for row in dataset:
	prediction = predict(stump, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))


Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
